# Programming and Database Fundamentals for Data Scientists - EAS503

The goal of PyMySQL is to be a drop-in replacement for MySQLdb and work on CPython, PyPy and IronPython.

### Installation
```script
pip install PyMySQL
```
or
```script
conda install PyMySQL
```
Might need `sudo` privileges depending on your Python installation.


### Start with a simple table
Run the following in your database
```sql
CREATE TABLE `users` (
    `id` int(11) NOT NULL AUTO_INCREMENT,
    `email` varchar(255) COLLATE utf8_bin NOT NULL,
    `password` varchar(255) COLLATE utf8_bin NOT NULL,
    PRIMARY KEY (`id`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
AUTO_INCREMENT=1 ;
```

In [2]:
import pymysql.cursors

In [4]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='eas503db',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
try:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "INSERT INTO `users` (`email`, `password`) VALUES (%s, %s)"
        cursor.execute(sql, ('admin@python.org', 'very-very-secret'))
        #sql = "INSERT INTO `users` (`email`, `password`) VALUES ('admin@python.org', 'very-very-secret')"
        #cursor.execute(sql)
    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

    
finally:
    connection.close()

In [40]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='eas503db',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
try:
    with connection.cursor() as cursor:
        # Create a new record
        sql = "DELETE FROM `users`"
        cursor.execute(sql)
    # connection is not autocommit by default. So you must commit to save
    # your changes.
    connection.commit()

    
finally:
    connection.close()

In [7]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='eas503db',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
try:
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT `id`, `password`, `email` FROM `users`"
        cursor.execute(sql)
        #result = cursor.fetchone()
        result = cursor.fetchall()
        print(result)
        
finally:
    connection.close()
        

[{'id': 1, 'password': 'very-very-secret', 'email': 'admin@python.org'}, {'id': 2, 'password': 'very-very-secret', 'email': 'admin@python.org'}]


### Using Pandas library

In [8]:
import pandas as pd

In [9]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='eas503db',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
df = pd.read_sql('SELECT * FROM users', con=connection)

In [10]:
df

,id,email,password
0,1,admin@python.org,very-very-secret
1,2,admin@python.org,very-very-secret


## Querying HR database

In [11]:
import time

In [12]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='employees',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [13]:
st = time.time()
df = pd.read_sql('''SELECT E.first_name,E.last_name,Y.salary
 FROM employees E JOIN (
  SELECT S.emp_no,S.salary
        FROM salaries S JOIN (
                SELECT emp_no,MAX(from_date) AS from_date
                FROM salaries
                GROUP BY emp_no) AS X
                ON S.emp_no = X.emp_no AND S.from_date = X.from_date
              ) Y
ON E.emp_no = Y.emp_no
WHERE E.hire_date >= \'2000-01-01\' ''', con=connection)
en = time.time()
print(en-st)

2.127168893814087


In [14]:
df

,first_name,last_name,salary
0,Ulf,Flexer,52206
1,Seshu,Rathonyi,61320
2,Randi,Luit,44619
3,Mariangiola,Boreale,62286
4,Ennio,Alblas,80596
5,Volkmar,Perko,52953
6,Xuejun,Benzmuller,53036
7,Shahab,Demeyer,43267
8,Jaana,Verspoor,40638
9,Jeong,Boreale,42176


In [15]:
connection.close()

## Alternative
Pull tables employees and salaries into Pandas and then use `pd.join()` and additional operations to get the above dataframe

In [64]:
st = time.time()
df = pd.read_sql('''SELECT E.first_name,E.last_name,E.hire_date,Y.salary
 FROM employees E JOIN (
  SELECT S.emp_no,S.salary
        FROM salaries S JOIN (
                SELECT emp_no,MAX(from_date) AS from_date
                FROM salaries
                GROUP BY emp_no) AS X
                ON S.emp_no = X.emp_no AND S.from_date = X.from_date
              ) Y
ON E.emp_no = Y.emp_no ''', con=connection)
en = time.time()
print(en-st)

31.47672700881958


In [63]:
df.head()

,first_name,last_name,salary
0,Georgi,Facello,88958
1,Bezalel,Simmel,72527
2,Parto,Bamford,43311
3,Chirstian,Koblick,74057
4,Kyoichi,Maliniak,94692


Find first_name, last_name of all employees whose current salary exceeds the current salary of their department managers

```sql
CREATE VIEW current_department_manager AS
SELECT D.emp_no, D.dept_no
FROM dept_manager D JOIN (
    SELECT emp_no,MAX(from_date) AS from_date
    FROM dept_manager
    GROUP BY emp_no) AS X
ON D.emp_no = X.emp_no AND D.from_date = X.from_date;
```

```sql
CREATE VIEW current_department_emp AS
SELECT D.emp_no, D.dept_no
FROM dept_emp D JOIN (
    SELECT emp_no,MAX(from_date) AS from_date
    FROM dept_emp
    GROUP BY emp_no) AS X
ON D.emp_no = X.emp_no AND D.from_date = X.from_date;
```

```sql
CREATE VIEW current_salary AS
SELECT S.emp_no, S.salary
FROM salaries S JOIN (
    SELECT emp_no,MAX(from_date) AS from_date
    FROM salaries
    GROUP BY emp_no) AS X
ON S.emp_no = X.emp_no AND S.from_date = X.from_date;
```